# Fetch raw data and validate

In [1]:
from pathlib import Path
import logging
from io import StringIO
import pandas as pd
from pandas.tseries.offsets import MonthEnd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromiumService
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
import os

def download_one_file_of_raw_data(year: int, month: int) -> Path:
    """
    This function takes a year and a month parameter and downloads the 
    NB Power System Information for the chosen year-month as a parquet file
    """
    
    # Chrome driver settings
    option = webdriver.ChromeOptions()
    option.add_argument("start-maximized")
    prefs = {'download.default_directory' : os.getcwd()+'/data/raw'}
    option.add_experimental_option('prefs', prefs)

    URL = 'https://tso.nbpower.com/Public/en/system_information_archive.aspx'
    
    # Validate parameter types
    if not isinstance(year, int):
          raise TypeError("Integer value required for Year parameter" )
    if not isinstance(month, int):
          raise TypeError("Integer value required for Month parameter" )
    
    # Configure logger
    logger = logging.getLogger('NB_Power_Extract')
    logger.setLevel(logging.DEBUG)

    # Create a file handler
    fh = logging.FileHandler(f'../log/extract_nb_power_{year}_{month:02d}.log')
    fh.setLevel(logging.DEBUG)

    # Create a console handler
    ch = logging.StreamHandler()
    ch.setLevel(logging.ERROR)

    # Create a formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)

    # Add handlers to the logger
    logger.addHandler(fh)
    logger.addHandler(ch)
    
    logger.info('Start Extract Session')
    try:
        # Initialize Chrome WebDriver 
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=option)
        
        # Load page
        driver.get(URL)
    
        # Set menus
        month_sel = driver.find_element("xpath",'/html/body/form/div[5]/div[1]/table[1]/tbody/tr[2]/td[1]/select')
        year_sel = driver.find_element("xpath",'//*[@id="ctl00_cphMainContent_ddlYear"]')
      
        # Use input parameters to select year
        year_dict = {val.text: index for index,val in enumerate(year_sel.find_elements(By.TAG_NAME,"option"))}
        year_menu = year_sel.find_elements(By.TAG_NAME,"option")
        year_menu[year_dict[str(year)]].click()
    
        # Use input parameters to select month
        month_dict = {val.text: index for index,val in enumerate(month_sel.find_elements(By.TAG_NAME,"option"))}
        month_menu = month_sel.find_elements(By.TAG_NAME,"option")
        month_menu[month_dict[str(month)]].click()
    
        # View data
        view_data_button = driver.find_element("xpath",'//*[@id="ctl00_cphMainContent_lbGetData"]')
        view_data_button.click()
    
        # Parse data into data frame then write to parquet
        output_text=driver.find_element(By.TAG_NAME,"pre").text
        csvStringIO = StringIO(output_text)
        df = pd.read_csv(csvStringIO, sep=",", header=0)
        
        path = f'../data/raw/nb_power_{year}_{month:02d}.parquet'
        df.to_parquet(path)
        logger.info('Extract completed!')
        return path

    
    except KeyError:
        # Close the WebDriver
        driver.quit()
        print("Invalid year and month parameters selected")
        logger.error('Invalid year and month parameters selected')
        
    except WebDriverException:
        # Close the WebDriver
        driver.quit()
        print("Base URL no longer valid - leads to NoSuchElementException")
        logger.error('Base URL no longer valid ')
        
    except Exception as e:
        # Close the WebDriver
        driver.quit()
        
        # Log the exception along with additional information
        logging.error('An error occurred: %s', str(e))
        print('An error occurred: %s', str(e))
    
    else:
        # Close the WebDriver
        driver.quit()

In [2]:
download_one_file_of_raw_data(year=2023, month=1)

'../data/raw/nb_power_2023_01.parquet'

In [3]:
download_one_file_of_raw_data(year=2025, month=1)

2024-02-18 20:42:18,002 - NB_Power_Extract - ERROR - Invalid year and month parameters selected
2024-02-18 20:42:18,002 - NB_Power_Extract - ERROR - Invalid year and month parameters selected


Invalid year and month parameters selected


In [3]:
download_one_file_of_raw_data(year='2020', month=1)

TypeError: Integer value required for Year parameter

In [4]:
download_one_file_of_raw_data(year=2020, month='1')

TypeError: Integer value required for Month parameter

In [5]:
df = pd.read_parquet('../data/raw/nb_power_2023_01.parquet')

df.head()

,HOUR,NB_LOAD,NB_DEMAND,ISO_NE,NMISA,QUEBEC,NOVA_SCOTIA,PEI
0,2023-01-01 00:00,1502,1572,196.0,42,-920,79,153
1,2023-01-01 01:00,1458,1525,108.0,40,-909,75,153
2,2023-01-01 02:00,1434,1498,115.0,41,-910,74,145
3,2023-01-01 03:00,1417,1482,113.0,39,-910,74,139
4,2023-01-01 04:00,1408,1473,203.0,38,-910,74,144


In [6]:
df['HOUR'] = pd.to_datetime(df['HOUR'])

df.HOUR.describe()

count                              744
mean     2023-01-16 11:29:59.999999744
min                2023-01-01 00:00:00
25%                2023-01-08 17:45:00
50%                2023-01-16 11:30:00
75%                2023-01-24 05:15:00
max                2023-01-31 23:00:00
Name: HOUR, dtype: object

In [7]:
pd.to_datetime(df['HOUR']).describe()

count                              744
mean     2023-01-16 11:29:59.999999744
min                2023-01-01 00:00:00
25%                2023-01-08 17:45:00
50%                2023-01-16 11:30:00
75%                2023-01-24 05:15:00
max                2023-01-31 23:00:00
Name: HOUR, dtype: object

In [8]:
def validate_raw_data(df: pd.DataFrame, year: int, month: int) -> Path:
    """
    """
    start_nrows = df.shape[0]
    month_end = (pd.to_datetime(f"{year}-{month:02d}-01",format="%Y-%m-%d") + MonthEnd(0)).date()
    
    df['HOUR'] = pd.to_datetime(df['HOUR'])
    df = df[df['HOUR'].apply(lambda x: x.date() <= month_end)]
    
    end_nrows = df.shape[0]
    rows_removed = start_nrows - end_nrows
    if start_nrows == end_nrows:
        print("Timestamps validated!")
        
    else:
        print(f"{rows_removed} removed to clean up timestamps")
    
    output_path = f'../data/transformed/nb_power_cleaned_{year}-{month:02d}.parquet'
    df.to_parquet(output_path)
    
    return output_path

In [9]:
validate_raw_data(df=df,year=2023,month=1)

Timestamps validated!


'../data/transformed/nb_power_cleaned_2023-01.parquet'